In [29]:
import os
from dotenv import load_dotenv
load_dotenv()


GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")


os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
#os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [30]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="llama3-groq-70b-8192-tool-use-preview")
llm.invoke("hi").content

'Hello! How can I assist you today?'

In [31]:
message=[{"role":"system","content":"you are a helpful assistant"},
         {"role":"user","content":"hi how are you?"}]

result = llm.invoke(message)
print(result.content)

I'm doing great, thanks! How can I assist you today?


In [32]:
class Chatbot:
    def __init__(self,system=""):
        self.system=system
        self.message=[]
        if self.system:
            self.message.append({"role":"system","content":system})
    def __call__(self,message):
        self.message.append({"role":"user","content":message})
        result=self.execute()
        self.message.append({"role":"assistant","content":result})
        return result
        
    def execute(self):
        llm = ChatGroq(model_name="Gemma2-9b-It")
        result = llm.invoke(self.message)
        return result.content
        
    


In [33]:
bot=Chatbot("you are a helpful assistant")

In [34]:
bot("hi")

"Hello! I'm glad you think so. \n\nHow can I help you today? 😊 \n\n"

In [35]:
bot.message

[{'role': 'system', 'content': 'you are a helpful assistant'},
 {'role': 'user', 'content': 'hi'},
 {'role': 'assistant',
  'content': "Hello! I'm glad you think so. \n\nHow can I help you today? 😊 \n\n"}]

In [36]:
bot.execute()

'Do you have any questions I can answer, tasks I can complete, or just want to chat?\n'

In [38]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

## Understand the Regular expression

Pattern Breakdown:

- ^: This matches the start of a string. It means the string must begin with what follows.
- Action:: This is a literal match. It means the string must have the text "Action:" at the beginning.
- (\w+):
    1. The parentheses () define a capture group. This allows you to extract part of the string that matches this section.
    2. \w+ matches one or more word characters (letters, digits, and underscores). This will capture a word that follows "Action:".
- :: This is a literal colon that separates the word matched by (\w+) from the rest of the string.
- (.*):
    1. This is another capture group, where .* matches any character (.) zero or more times (*), which means it captures everything that comes after the second colon.

### What does it do?
This regex is looking for a string that:

1. Starts with the text "Action:".
2. Has a word right after it, separated by a colon.
3. Then, after another colon, it captures everything that follows.

In [39]:
import re
action_re = re.compile('^Action: (\w+): (.*)')

In [ ]:
import re

# Compiling the regular expression pattern
action_re = re.compile('^Action: (\w+): (.*)')

# Sample strings
text1 = "Action: Move: North"
text2 = "Action: Jump: High"
text3 = "Error: Not an Action"

In [ ]:
# Testing the pattern
match1 = action_re.match(text1)

In [ ]:
match1

In [ ]:
match1.group(1)

In [ ]:
match1.group(2)

In [ ]:
match2 = action_re.match(text2)

In [ ]:
match2

In [ ]:
match3 = action_re.match(text3)

In [ ]:
match3

In [ ]:
# Extracting the matched groups
if match1:
    print(f"Matched: {match1.group(1)}, {match1.group(2)}")

In [ ]:
if match2:
    print(f"Matched: {match2.group(1)}, {match2.group(2)}")  # Jump, High

In [ ]:
if match3:
    print("No match")  # This won't match, as it doesn't start with "Action:"

In [40]:
import httpx
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]


In [41]:
import httpx
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]

In [42]:
def calculate(number):
    return eval(number)

In [43]:
calculate("2+2")

4

In [44]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [51]:
bot = Chatbot(prompt)

In [52]:
next_prompt="Tell me about quantum computing from Wikipedia."

In [53]:
result = bot(next_prompt)

In [54]:
result

'Thought: I will ask Wikipedia for a summary of quantum computing. \n\nAction: wikipedia: Quantum computing \nPAUSE \n'

In [55]:
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

In [56]:
actions

[<re.Match object; span=(0, 37), match='Action: wikipedia: Quantum computing '>]

In [57]:
action, action_input = actions[0].groups()

In [59]:
action_input

'Quantum computing '

In [58]:
action

'wikipedia'

In [60]:
known_actions[action]

<function __main__.wikipedia(query)>

In [61]:
known_actions[action](action_input)

'waves, and <span class="searchmatch">quantum</span> <span class="searchmatch">computing</span> leverages this behavior using specialized hardware. Classical physics cannot explain the operation of these <span class="searchmatch">quantum</span> devices'

In [45]:
def query(question,max_turns=5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result

In [46]:
query("hello")

Understood. I'm ready for my first question.  



"Understood. I'm ready for my first question.  \n"

In [47]:
query("how are you?")

Okay. I am good.  

How can I help you?



'Okay. I am good.  \n\nHow can I help you?\n'

In [48]:
query("what you doing?")

Understood.  

Thought:  I'm waiting for a question to answer. 

Action:  
PAUSE 



"Understood.  \n\nThought:  I'm waiting for a question to answer. \n\nAction:  \nPAUSE \n"

In [49]:
print(query("Fifteen * twenty five"))

Thought: I need to calculate 15 * 25.

Action: calculate: 15 * 25 
PAUSE 

 -- running calculate 15 * 25 
Observation: 375
Answer: 375 

Answer: 375 



In [50]:
query("Has Simon written about AI?")

Thought: I can search Simon's blog to see if he has written about AI. 
Action: simon_blog_search: AI
PAUSE 

 -- running simon_blog_search AI
Observation: It's OK to call it Artificial Intelligence: Update 9th January 2024: This post was clumsily written and failed to make the point I wanted it to make. I've published a follow-up, What I should have said about the term Artificial Intelligence which you should read instead.
My original post follows.


We need to be having high quality conversations about AI: what it can and can't do, its many risks and pitfalls and how to integrate it into society in the most beneficial ways possible.
Any time I write anything that mentions AI it's inevitable that someone will object to the very usage of the term.
Strawman: "Don't call it AI! It's not actually intelligent - it's just spicy autocomplete."
That strawman is right: it's not "intelligent" in the same way that humans are. And "spicy autocomplete" is actually a pretty good analogy for how a lo

'Answer: Yes, Simon has written about AI. \n'